In [1]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install -U pysal
!pip install geopandas
!pip install torch_geometric pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cpu.html
!pip install torch-geometric-temporal
!pip install --upgrade --force-reinstall git+https://github.com/FelipeSchreiber/COE770_Machine_Learning_on_Graphs.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.pyg.org/whl/torch-2.0.0+cpu.html
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/FelipeSchreiber/COE770_Machine_Learning_on_Graphs.git to /tmp/pip-req-build-__52c6es
  Running command git clone --filter=blob:none --quiet https://github.com/FelipeSchreiber/COE770_Machine_Learning_on_Graphs.git /tmp/pip-req-build-__52c6es
  Resolved https://github.com/FelipeSchreiber/COE770_Machine_Learning_on_Graphs.git to commit be88f9d0976b7b79d4d667f8d9d3a4cc518f7def
  Preparing metadata (setup.py) ... 

In [3]:
print("Ok")

Ok


In [4]:
from google.colab import runtime
import torch
import gc
from dataset.covidBR_dataset import *
from model.covid_model import *
from torch_geometric_temporal.signal import temporal_signal_split
from torch_geometric.loader import GraphSAINTRandomWalkSampler, NeighborLoader
from tqdm import tqdm

In [5]:
def free_cache():
  gc.collect()
  torch.cuda.empty_cache()

In [6]:
loader = CovidDatasetLoader(method="other")
dataset = loader.get_dataset(lags=4)
train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio=0.8)

onehot:  (5570, 27)
centroids:  (5570, 2)


/usr/local/lib/python3.10/dist-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 3 disconnected components.
 There are 2 islands with ids: 1525, 3499.
  warnings.warn(message)
/usr/local/lib/python3.10/dist-packages/libpysal/weights/weights.py:172: UserWarning: The weights matrix is not fully connected: 
 There are 1421 disconnected components.
  warnings.warn(message)


[]


/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


Index(['city_ibge_code', 'date', 'epidemiological_week',
       'estimated_population_2019',
       'last_available_confirmed_per_100k_inhabitants',
       'last_available_death_rate', 'last_available_deaths', 'order_for_place',
       'new_confirmed', 'new_deaths'],
      dtype='object')
feat:  757 (5570, 35)


In [7]:
dataset[0].x.shape[1]

35

In [8]:
filter_size = 3
train_model = False
num_feats = dataset[0].x.shape[1]
gamma=1

In [9]:
device = "cpu"
dtype = torch.FloatTensor
if torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor
    device = "cuda"
if device == "cpu" and train_model:
    import torch_directml
    dml = torch_directml.device()
device

'cuda'

In [10]:
if train_model:
    model = RecurrentGCN(node_features = num_feats, num_filters = filter_size).to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

    model.train()

    for epoch in tqdm(range(100)):
        for time, snapshot in enumerate(train_dataset):
            snapshot.to(device)
            y_hat,A_h,A_z,A_r  = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
            cost = torch.mean((y_hat-snapshot.y)**2)\
            + gamma*(torch.norm(A_h,p=1) + torch.norm(A_z,p=1) + torch.norm(A_r,p=1))
            cost.backward()
            optimizer.step()
            optimizer.zero_grad()
        if epoch % 10 == 0:
            torch.save(model.state_dict(), "./model_weights")
            # torch.save(model, "./the_whole_model")
    torch.save(model, "./the_whole_model")
    free_cache()
else:
    model = get_model(False)
    model.to(device)

Downloading...
From: https://drive.google.com/uc?export=download&id=1skGodpbZZhuDyvxJbicnIe8ed9d0w5iN
To: /content/covid_model_weights
100%|██████████| 5.83k/5.83k [00:00<00:00, 21.4MB/s]


In [11]:
next(model.parameters()).is_cuda

True

In [12]:
def check_mem():
  #Additional Info when using cuda
  if device == 'cuda':
      print(torch.cuda.get_device_name(0))
      print('Memory Usage:')
      print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
      print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

In [13]:
check_mem()

Tesla T4
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


/usr/local/lib/python3.10/dist-packages/torch/cuda/memory.py:416: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [14]:
# free_cache()

In [ ]:
model.eval()
cost = 0
# test_dataset.to(device)
for time, snapshot in enumerate(test_dataset):
    snapshot.to(device)
    y_hat = model(snapshot.x, snapshot.edge_index, snapshot.edge_attr)
    cost = cost + torch.mean((y_hat-snapshot.y)**2).item()
    cost = cost / (time+1)
    del snapshot
    free_cache()
    # check_mem()

In [19]:
print("MSE: {:.4f}".format(cost))

MSE: 1003601.3972


In [18]:
time

151

In [ ]:
runtime.unassign()